In [1]:
import pickle
import numpy as np
import cv2
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

from keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.models import Model
from sklearn.metrics import confusion_matrix, roc_curve, precision_recall_curve, auc

In [2]:
import random
import os

def set_seed(seed=42):

    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    
    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['TF_DETERMINISTIC_OPS'] = '1'

set_seed(42)

## Model

In [3]:
'''
Script: resnet_builder_keras.py
Dependency environment: tf_gpu

Script for building 3D Resnet models in pure Keras. 
Code adapted from https://github.com/JihongJu/keras-resnet3d updated for TF2.0
'''

from __future__ import (
    absolute_import,
    division,
    print_function,
    unicode_literals
)
import six
from math import ceil
import keras
from keras.models import Model
from keras.layers import (
    Input,
    Activation,
    Dense,
    Flatten,
    Add
)

from keras.layers import (
    Conv3D,
    AveragePooling3D,
    MaxPooling3D,
    Dropout
)

from keras.layers import concatenate
from keras.layers import BatchNormalization
from keras.regularizers import l2
from keras import backend as K


def _bn_relu(input):
    """Helper to build a BN (non-trainable) -> relu block."""
    norm = BatchNormalization(axis=CHANNEL_AXIS, trainable=False)(input)  # Set trainable to False
    return Activation("relu")(norm)

def _conv_bn_relu3D(**conv_params):
    filters = conv_params["filters"]
    kernel_size = conv_params["kernel_size"]
    strides = conv_params.setdefault("strides", (1, 1, 1))
    kernel_initializer = conv_params.setdefault(
        "kernel_initializer", "he_normal")
    padding = conv_params.setdefault("padding", "same")
    kernel_regularizer = conv_params.setdefault("kernel_regularizer",
                                                l2(1e-3))

    def f(input):
        conv = Conv3D(filters=filters, kernel_size=kernel_size,
                      strides=strides, kernel_initializer=kernel_initializer,
                      padding=padding,
                      kernel_regularizer=kernel_regularizer)(input)
        return _bn_relu(conv)

    return f

def _bn_relu_conv3d(**conv_params):
    """Helper to build a BN -> relu -> Dropout -> conv3d block."""
    filters = conv_params["filters"]
    kernel_size = conv_params["kernel_size"]
    strides = conv_params.setdefault("strides", (1, 1, 1))
    kernel_initializer = conv_params.setdefault("kernel_initializer", "he_normal")
    padding = conv_params.setdefault("padding", "same")
    kernel_regularizer = conv_params.setdefault("kernel_regularizer", l2(1e-3))
    dropout_rate = conv_params.setdefault("dropout_rate", 0.3)  # Set the dropout rate here

    def f(input):
        activation = _bn_relu(input)
        dropout = Dropout(rate=dropout_rate)(activation)  # Apply dropout after activation
        conv = Conv3D(
            filters=filters,
            kernel_size=kernel_size,
            strides=strides,
            kernel_initializer=kernel_initializer,
            padding=padding,
            kernel_regularizer=kernel_regularizer
        )(dropout)
        return conv
    
    return f


def _shortcut3d(input, residual):
    """3D shortcut to match input and residual and merges them with "sum"."""
    stride_dim1 = ceil(int(input.shape[DIM1_AXIS]) \
        / float(int(residual.shape[DIM1_AXIS])))
    stride_dim2 = ceil(int(input.shape[DIM2_AXIS]) \
        / float(int(residual.shape[DIM2_AXIS])))
    stride_dim3 = ceil(int(input.shape[DIM3_AXIS]) \
        / float(int(residual.shape[DIM3_AXIS])))
    equal_channels = residual.shape[CHANNEL_AXIS] \
        == input.shape[CHANNEL_AXIS]

    shortcut = input
    if stride_dim1 > 1 or stride_dim2 > 1 or stride_dim3 > 1 \
            or not equal_channels:
        shortcut = Conv3D(
            filters=residual.shape[CHANNEL_AXIS],
            kernel_size=(1, 1, 1),
            strides=(stride_dim1, stride_dim2, stride_dim3),
            kernel_initializer="he_normal", padding="valid",
            kernel_regularizer=l2(1e-4)
            )(input)
    return Add()([shortcut, residual])


def _residual_block3d(block_function, filters, kernel_regularizer, repetitions,
                      is_first_layer=False):
    def f(input):
        for i in range(repetitions):
            strides = (1, 1, 1)
            if i == 0 and not is_first_layer:
                strides = (2, 2, 2)
            input = block_function(filters=filters, strides=strides,
                                   kernel_regularizer=kernel_regularizer,
                                   is_first_block_of_first_layer=(
                                       is_first_layer and i == 0)
                                   )(input)
        return input

    return f


def basic_block(filters, strides=(1, 1, 1), kernel_regularizer=l2(1e-4),
                is_first_block_of_first_layer=False):
    """Basic 3 X 3 X 3 convolution blocks. Extended from raghakot's 2D impl."""
    def f(input):
        if is_first_block_of_first_layer:
            # don't repeat bn->relu since we just did bn->relu->maxpool
            conv1 = Conv3D(filters=filters, kernel_size=(3, 3, 3),
                           strides=strides, padding="same",
                           kernel_initializer="he_normal",
                           kernel_regularizer=kernel_regularizer
                           )(input)
        else:
            conv1 = _bn_relu_conv3d(filters=filters,
                                    kernel_size=(3, 3, 3),
                                    strides=strides,
                                    kernel_regularizer=kernel_regularizer
                                    )(input)

        residual = _bn_relu_conv3d(filters=filters, kernel_size=(3, 3, 3),
                                   kernel_regularizer=kernel_regularizer
                                   )(conv1)
        return _shortcut3d(input, residual)

    return f


def _handle_data_format():
    global DIM1_AXIS
    global DIM2_AXIS
    global DIM3_AXIS
    global CHANNEL_AXIS
    if K.image_data_format() == 'channels_last':
        DIM1_AXIS = 1
        DIM2_AXIS = 2
        DIM3_AXIS = 3
        CHANNEL_AXIS = 4
    else:
        CHANNEL_AXIS = 1
        DIM1_AXIS = 2
        DIM2_AXIS = 3
        DIM3_AXIS = 4


def _get_block(identifier):
    if isinstance(identifier, six.string_types):
        res = globals().get(identifier)
        if not res:
            raise ValueError('Invalid {}'.format(identifier))
        return res
    return identifier


class Resnet3DBuilder(object):
    """ResNet3D."""

    @staticmethod
    def build(input_shape, num_outputs, block_fn, repetitions, reg_factor):
        """Instantiate a vanilla ResNet3D keras model.
        # Arguments
            input_shape: Tuple of input shape in the format
            (conv_dim1, conv_dim2, conv_dim3, channels) if dim_ordering='
            (filter, conv_dim1, conv_dim2, conv_dim3) if dim_ordering='th'
            num_outputs: The number of outputs at the final softmax layer
            block_fn: Unit block to use {'basic_block', 'bottlenack_block'}
            repetitions: Repetitions of unit blocks
        # Returns
            model: a 3D ResNet model that takes a 5D tensor (volumetric images
            in batch) as input and returns a 1D vector (prediction) as output.
        """
        _handle_data_format()
        if len(input_shape) != 4:
            raise ValueError("Input shape should be a tuple "
                             "(conv_dim1, conv_dim2, conv_dim3, channels) "
                             "for tensorflow as backend or "
                             "(channels, conv_dim1, conv_dim2, conv_dim3) "
                             "for theano as backend")

        block_fn = _get_block(block_fn)
        input = Input(shape=input_shape)
        
        # first conv
        conv1 = _conv_bn_relu3D(filters=64, kernel_size=(7, 7, 7),
                                strides=(2, 2, 2),
                                kernel_regularizer=l2(reg_factor)
                                )(input)
        pool1 = MaxPooling3D(pool_size=(3, 3, 3), strides=(1, 1, 1),
                             padding="same")(conv1)

        # repeat blocks
        block = pool1
        filters = 64
        for i, r in enumerate(repetitions):
            block = _residual_block3d(block_fn, filters=filters,
                                      kernel_regularizer=l2(reg_factor),
                                      repetitions=r, is_first_layer=(i == 0)
                                      )(block)
            filters *= 2

        # last activation
        block_output = _bn_relu(block)

        # average poll and classification
        pool2 = AveragePooling3D(pool_size=(block.shape[DIM1_AXIS],
                                            block.shape[DIM2_AXIS],
                                            block.shape[DIM3_AXIS]),
                                 strides=(1, 1, 1))(block_output)
        flatten1 = Flatten()(pool2)
        
        if num_outputs > 1:
            dense = Dense(units=num_outputs,
                          kernel_initializer="he_normal",
                          activation="softmax",
                          kernel_regularizer=l2(reg_factor))(flatten1)
        else:
            dense = Dense(units=num_outputs,
                          kernel_initializer="he_normal",
                          activation="sigmoid",
                          kernel_regularizer=l2(reg_factor))(flatten1)

        model = Model(inputs=input, outputs=dense)
        return model

    @staticmethod
    def build_resnet_18(input_shape, num_outputs, reg_factor=2e-4):
        """Build resnet 18."""
        return Resnet3DBuilder.build(input_shape, num_outputs, basic_block,
                                     [2, 2, 2, 2], reg_factor=reg_factor)

In [4]:
from keras.layers import Dense, Input, concatenate
from keras.models import Model

def resnet18_(pixel=56, num_outputs=2):
    
    # Instantiate the models for each branch
    optical_flow_model = Resnet3DBuilder.build_resnet_18(input_shape=(37, pixel, pixel, 2), num_outputs=num_outputs)
    #optical_flow_model.summary()
    
    image_model = Resnet3DBuilder.build_resnet_18(input_shape=(38, pixel, pixel, 1), num_outputs=num_outputs)
    #image_model.summary()
    
    # Create input layers
    optical_flow_input = optical_flow_model.input
    image_input = image_model.input
    
    # Assuming you want to remove the last layer and use the penultimate layer's output
    optical_flow_output = optical_flow_model.layers[-2].output
    image_output = image_model.layers[-2].output
    
    # Concatenate the outputs of the two branches
    combined = concatenate([image_output, optical_flow_output])
    
    # Define the logits layer explicitly
    logits = Dense(num_outputs, activation=None, name='logits')(combined)
    
    # Add activation to the logits for final predictions
    predictions = tf.keras.layers.Activation('sigmoid' if num_outputs == 1 else 'softmax', name='predictions')(logits)
    model = Model(inputs=[image_input, optical_flow_input], outputs=predictions)

    #model.summary()

    return model

## Evaluating model

In [5]:
def load_model_for_fold(weights_path, model, num_outputs=2):

    """
    Load and return a model with freshly initialized fully connected layers for each fold.


    Parameters:
    - weights_path: Path to the weights file for the fold.
    - model: The base model without the final fully connected layers.
    - num_outputs: Number of output classes.


    Returns:
    - conv_model: Model with newly initialized fully connected layers and loaded weights.
    """
    
    base_output = model.layers[-3].output  # Output from the layer before the FC layer
    logits = Dense(num_outputs, activation=None, name='logits_new')(base_output)
    predictions = Activation('sigmoid' if num_outputs == 1 else 'softmax', name='predictions_new')(logits)
    conv_model = Model(inputs=model.input, outputs=predictions)

    conv_model.load_weights(weights_path)

    conv_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    return conv_model



def evaluate_fold_model(conv_model, X_test, y_test, fold_idx):

    """

    Evaluate the model for a specific fold, compute metrics (including confusion matrix, ROC AUC, PR AUC), 
    plot ROC and PR curves, and save predicted probabilities in a .txt file.

    Parameters:
    - conv_model: The model to be evaluated.
    - X_test: Test features (which could include optical flow or not).
    - y_test: True labels (one-hot encoded).
    - fold_idx: Index of the fold being evaluated (for saving the output).

    
    Returns:
    - metrics_dict: Dictionary of accuracy, loss, ROC AUC, PR AUC.
    - y_pred_probs: Predicted probabilities for the test set.
    """

    # Handle cases where no optical flow data is provided

    if X_test[1] is None:
        X_test = X_test[0]  # Only use the main test data if optical flow is not provided

    # Evaluate the model on the test set
    loss, accuracy = conv_model.evaluate(X_test, y_test, verbose=0)

    # Predict probabilities
    y_pred_probs = conv_model.predict(X_test, verbose=0)

    # Convert probabilities to predicted class indices
    y_pred_classes = np.argmax(y_pred_probs, axis=1)
    y_true_classes = np.argmax(y_test, axis=1)

    # Compute the confusion matrix
    conf_matrix = confusion_matrix(y_true_classes, y_pred_classes)

    # Compute ROC AUC and PR AUC with class 1 probabilities
    prob_class_1 = y_pred_probs[:, 1]


    # ROC Curve and AUC computation
    fpr, tpr, roc_thresholds = roc_curve(y_true_classes, prob_class_1)
    #print("fpr", fpr)
    #print("tpr", tpr)
    #print("roc_thresholds",roc_thresholds)
    roc_auc = auc(fpr, tpr)

    # Precision-Recall Curve and AUC computation
    precision, recall, pr_thresholds = precision_recall_curve(y_true_classes, prob_class_1)
    #print("precision", precision)
    #print("recall", recall)
    #print("pr_thresholds",pr_thresholds)
    pr_auc = auc(recall, precision)
    
    return loss, accuracy, roc_auc, pr_auc, conf_matrix

In [6]:
def load_fold_data(fold_dir, k_fold):

    """
    Load the training and validation data for a specific fold.
    Parameters:
    
    - fold_dir: Directory where the fold's data is stored.
    - k_fold: The fold number to load the data from.

    Returns:
    - X_train_fold, X_val_fold: Training and validation image data.
    - y_train_fold, y_val_fold: Training and validation label data.
    - optical_flow_train_fold, optical_flow_val_fold: Optical flow data for training and validation.
    """

    X_train_fold = np.load(os.path.join(fold_dir, f"fold_{k_fold}/train_videos.npy"))
    X_val_fold = np.load(os.path.join(fold_dir, f"fold_{k_fold}/test_videos.npy"))
    y_train_fold = np.load(os.path.join(fold_dir, f"fold_{k_fold}/train_labels.npy"))
    y_val_fold = np.load(os.path.join(fold_dir, f"fold_{k_fold}/test_labels.npy"))
    optical_flow_train_fold = np.load(os.path.join(fold_dir, f"fold_{k_fold}/train_optical_flow.npy"))
    optical_flow_val_fold = np.load(os.path.join(fold_dir, f"fold_{k_fold}/test_optical_flow.npy"))

    return X_train_fold, X_val_fold, y_train_fold, y_val_fold, optical_flow_train_fold, optical_flow_val_fold


def run_kfold_evaluation(weights_base_path, dropout_fcc, dropout_cnn, l1_reg, 
                         fold_dir = '/kaggle/input/13102024-5fold-splits-cross-validation/split_folds', model=resnet18_(), num_outputs=2, k=5):

    df_results = pd.DataFrame(columns=['data_type','dropout_fcc', 'l1_reg', 'dropout_cnn', 'fold', 
                                       'loss', 'accuracy', 'roc_auc', 'pr_auc', 'confusion_matrix'])
    

    for fold_idx in range(1, k + 1):
        
        print(f"Evaluating Fold {fold_idx}...")

        path = os.path.join(weights_base_path, f"Best_fold_{fold_idx}_cp.ckpt.weights.h5")

        conv_model = load_model_for_fold(path, model, num_outputs)

        X_train_fold, X_val_fold, y_train_fold, y_val_fold, optical_flow_train_fold, optical_flow_val_fold = load_fold_data(fold_dir, fold_idx)

        loss_train, accuracy_train, roc_auc_train, pr_auc_train, conf_matrix_train = evaluate_fold_model(conv_model, [X_train_fold, optical_flow_train_fold], y_train_fold, fold_idx)

        loss_val, accuracy_val, roc_auc_val, pr_auc_val, conf_matrix_val = evaluate_fold_model(conv_model, [X_val_fold, optical_flow_val_fold], y_val_fold, fold_idx)
        
        # Collect training metrics for each fold
        new_row_train = pd.DataFrame({
            'data_type' : ['train'],
            'dropout_fcc': [dropout_fcc],
            'l1_reg': [l1_reg],
            'dropout_cnn': ['N/A'],
            'fold': [fold_idx],
            'loss': [loss_train],
            'accuracy': [accuracy_train],
            'roc_auc': [roc_auc_train],
            'pr_auc': [pr_auc_train],
            'confusion_matrix':[conf_matrix_train]})
        
        df_results = pd.concat([df_results, new_row_train], ignore_index=True)

        # Collect validation metrics for each fold
        new_row_test = pd.DataFrame({
            'data_type' : ['test'],
            'dropout_fcc': [dropout_fcc],
            'l1_reg': [l1_reg],
            'dropout_cnn': ['N/A'],
            'fold': [fold_idx],
            'loss': [loss_val],
            'accuracy': [accuracy_val],
            'roc_auc': [roc_auc_val],
            'pr_auc': [pr_auc_val],
            'confusion_matrix':[conf_matrix_val]})

        df_results = pd.concat([df_results, new_row_test], ignore_index=True)
        
    return df_results

In [7]:
#df_results = run_kfold_evaluation(weights_base_path = '/kaggle/input/20102024-5fold-training-npv-loops-0-001-models/training_weights_dropout_0.3_l1_0.001', 
#                                 dropout_fcc = 0.3, 
#                                 dropout_cnn = 'N/A', 
#                                 l1_reg = 0.01)

In [8]:
#df_results.to_csv('model_results_test.csv', index=False)

## Loops


In [9]:
def evaluate_models_(l1_reg, dropout_fcc ,base_path):
    
    main_df = pd.DataFrame(columns=['data_type', 'dropout_fcc', 'l1_reg', 'dropout_cnn', 'fold', 
                                'loss', 'accuracy', 'roc_auc', 'pr_auc', 'confusion_matrix', 'model_type'])
    
        
        
    for j in range(len(dropout_fcc)):

        print(f'Hyperparams:= l1_ref: {l1_reg}, dropoutfcc: {dropout_fcc[j]}, dropoutcnn: N/A')

        model_path = os.path.join(base_path, f'training_weights_dropout_{dropout_fcc[j]}_l1_{l1_reg}')

        df_results = run_kfold_evaluation(weights_base_path = model_path, 
                                          dropout_fcc = dropout_fcc[j], 
                                          dropout_cnn = 'N/A', 
                                          l1_reg = l1_reg)

        main_df['model_type'] = 'pre-tunning'

        main_df = pd.concat([main_df, df_results], ignore_index=True)

        print(' ')
        
    return main_df

In [10]:
l1_reg = 0.1


dropout_fcc = [0.3,0.4,0.5,0.6,0.7,0.8,0.9]

In [11]:
base_path = f"/kaggle/input/20102024-5fold-training-npv-loops-0-1-models"

final_df = evaluate_models_(l1_reg, dropout_fcc ,base_path)

Hyperparams:= l1_ref: 0.1, dropoutfcc: 0.3, dropoutcnn: N/A
Evaluating Fold 1...


I0000 00:00:1730235337.938598      68 service.cc:145] XLA service 0x78f2bc132770 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1730235337.938661      68 service.cc:153]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1730235341.782598      68 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
/tmp/ipykernel_25/2174596720.py:60: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_results = pd.concat([df_results, new_row_train], ignore_index=True)


Evaluating Fold 2...
Evaluating Fold 3...
Evaluating Fold 4...
Evaluating Fold 5...


/tmp/ipykernel_25/3814762108.py:21: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  main_df = pd.concat([main_df, df_results], ignore_index=True)


 
Hyperparams:= l1_ref: 0.1, dropoutfcc: 0.4, dropoutcnn: N/A
Evaluating Fold 1...


...

Evaluating Fold 2...
Evaluating Fold 3...
Evaluating Fold 4...
Evaluating Fold 5...
 
Hyperparams:= l1_ref: 0.1, dropoutfcc: 0.8, dropoutcnn: N/A
Evaluating Fold 1...


/tmp/ipykernel_25/2174596720.py:60: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_results = pd.concat([df_results, new_row_train], ignore_index=True)


Evaluating Fold 2...
Evaluating Fold 3...
Evaluating Fold 4...
Evaluating Fold 5...
 
Hyperparams:= l1_ref: 0.1, dropoutfcc: 0.9, dropoutcnn: N/A
Evaluating Fold 1...


/tmp/ipykernel_25/2174596720.py:60: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_results = pd.concat([df_results, new_row_train], ignore_index=True)


Evaluating Fold 2...
Evaluating Fold 3...
Evaluating Fold 4...
Evaluating Fold 5...
 


In [12]:
final_df.to_csv('model_final_0.1.csv', index=False)